In [1]:
from pprint import pprint

import matplotlib.pyplot as plt
from braket.aws import AwsDevice
from braket.devices import Devices
from braket.jobs import hybrid_job, save_job_result
from qiskit.primitives import BackendEstimator

from qiskit_braket_provider import BraketProvider

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, QEOM, EvaluationRule
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

import time

sv1 = AwsDevice(Devices.Amazon.SV1)
image_uri2 = "011528273261.dkr.ecr.eu-west-2.amazonaws.com/amazon-braket-inhibitq-qiskit:latest"


In [2]:
@hybrid_job(device=sv1.arn, include_modules="qiskit_algorithms", image_uri=image_uri2)
def main():
    """Decorated function that will be run in the docker container."""

    # define problem observable, ansatz, vqe, qeom
    driver = PySCFDriver(
        atom="H 0 0 0; H 0 0 0.735",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )

    es_problem = driver.run()

    mapper = JordanWignerMapper()

    ansatz = UCCSD(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
        ),
    )

    #create estimator SV1
    backend = BraketProvider().get_backend("SV1")
    estimator = BackendEstimator(backend=backend, options={"shots": 1000})
    
    #estimator = Estimator()
    # This first part sets the ground state solver
    # see more about this part in the ground state calculation tutorial
    solver = VQE(estimator, ansatz, SLSQP())
    solver.initial_point = [0.0] * ansatz.num_parameters
    gse = GroundStateEigensolver(mapper, solver)

    # The qEOM algorithm is simply instantiated with the chosen ground state solver and Estimator primitive
    qeom_excited_states_solver = QEOM(gse, estimator, "sd", EvaluationRule.ALL)

    start = time.time()
    qeom_results = qeom_excited_states_solver.solve(es_problem)
    end = time.time()
   
    # Save the results of the VQE computation.
    save_job_result(
        {
            #"qeom": {
            #    "qeom_results": qeom_results,
            #},
            "time": {
                "start": start,
                "end": end,
                "diff": (end-start)
            }
        }
    )

Skipping python version validation, make sure versions match between local environment and container.


In [3]:
job = main()

In [4]:
result = job.result()
pprint(result)

{'time': {'diff': 1255.0657875537872,
          'end': 1734624609.3965135,
          'start': 1734623354.330726}}
